In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, f1_score
import numpy as np
import pandas as pd

**Loading Data**

In [5]:
TRAIN_PATH = "amazon_review_ID.shuf.lrn.csv"
TEST_PATH = "amazon_review_ID.shuf.tes.csv"
ID_COL = "ID"
TARGET_COL = "Class"
SUBMIT_OUT = "submission.csv"

df = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

print(df.shape)
df.head()

(750, 10002)


,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V9992,V9993,V9994,V9995,V9996,V9997,V9998,V9999,V10000,Class
0,0,17,4,8,8,9,4,0,2,3,...,0,0,0,0,0,0,0,1,1,Shea
1,1,21,9,5,8,6,2,16,3,12,...,0,0,0,2,2,1,0,1,0,Riley
2,2,9,7,6,3,8,2,9,4,4,...,0,0,0,0,0,0,0,1,1,Chachra
3,3,8,3,5,2,4,3,8,2,4,...,0,0,1,0,1,0,0,0,0,Agresti
4,4,15,8,8,4,7,8,4,7,1,...,0,0,0,0,0,0,0,0,0,Nigam


750 rows and 10,002 columns


**Pre-Processing**

In [6]:
# Basic hygiene
df.columns = df.columns.str.strip()
test.columns = test.columns.str.strip()

missing_train = df.isnull().sum()
n_missing_train = (missing_train > 0).sum()

print(f"Columns with missing values: {n_missing_train}")
print(f"Duplicate rows: {df.duplicated().sum()}")

constant_cols = [c for c in df.columns if df[c].nunique() == 1]
print(f"Constant columns: {len(constant_cols)}")

for col in df.select_dtypes(include=["int"]).columns:
    df[col] = pd.to_numeric(df[col], downcast="integer")


Columns with missing values: 0
Duplicate rows: 0
Constant columns (same value for all rows): 0


In [7]:
# Feature grouping
binary_thresh = 2
low_thresh = 10
nunique = df.nunique()

constant = nunique[nunique == 1].index.tolist()
binary = nunique[nunique == binary_thresh].index.tolist()
low_card = nunique[(nunique > binary_thresh) & (nunique <= low_thresh)].index.tolist()
continuous = nunique[nunique > low_thresh].index.tolist()

groups = {"constant": constant, "binary": binary, "categorical": low_card, "numeric": continuous}


In [8]:
le = LabelEncoder()
y = le.fit_transform(df[TARGET_COL])
X = df.drop(columns=[TARGET_COL, ID_COL], errors="ignore")
X_test_raw = test.drop(columns=[ID_COL], errors="ignore")

In [9]:
for key in groups:
    groups[key] = [col for col in groups[key] if col in X.columns]

preprocess = ColumnTransformer(
    transformers=[
        ("binary", "passthrough", groups["binary"]),
        ("categorical", OneHotEncoder(handle_unknown="ignore", sparse_output=False), groups["categorical"]),
        ("numeric", RobustScaler(), groups["numeric"])
    ],
    remainder="drop"
)

feature_selection = SelectKBest(score_func=mutual_info_classif, k=500)


In [ ]:
# Models to compare
models = {
    "LogReg": Pipeline([
        ("prep", preprocess),
        ("select", feature_selection),
        ("clf", LogisticRegression(
            penalty="elasticnet", solver="saga", class_weight="balanced",
            l1_ratio=0.5, C=0.5, max_iter=1000, n_jobs=-1, random_state=42
        ))
    ]),
    "SVM": Pipeline([
        ("prep", preprocess),
        ("select", feature_selection),
        ("clf", SVC(kernel="rbf", C=0.6, gamma="scale", probability=False, random_state=42))
    ]),
    "DecisionTree": Pipeline([
        ("prep", preprocess),
        ("select", feature_selection),
        ("clf", DecisionTreeClassifier(criterion="gini", max_depth=20, min_samples_split=10,
                                       min_samples_leaf=5, random_state=42))
    ])
}

scoring = {
    "accuracy": make_scorer(accuracy_score),
    "macro_f1": make_scorer(f1_score, average="macro")
}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Cross Validation
pd.set_option("display.precision", 4)

results = []
for name, pipe in models.items():
    cv_res = cross_validate(pipe, X, y, cv=cv, scoring=scoring, n_jobs=-1, return_train_score=False)
    results.append({
        "Model": name,
        "CV_Accuracy": np.mean(cv_res["test_accuracy"]),
        "CV_MacroF1": np.mean(cv_res["test_macro_f1"])
    })

res_df = pd.DataFrame(results).sort_values("CV_MacroF1", ascending=False)
print("\nModel selection (CV on training data):")
print(res_df.to_string(index=False))

best_name = res_df.iloc[0]["Model"]
print(f"\nSelected best model by CV Macro-F1: {best_name}")

best_pipeline = models[best_name]


**Random Forest**

In [14]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

scoring = {
    "accuracy": make_scorer(accuracy_score),
    "macro_f1": make_scorer(f1_score, average="macro")
}

# Define Random Forest pipeline
rf_pipeline = Pipeline([
    ("select", feature_selection),  # your existing SVD or feature selection step
    ("clf", RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        min_samples_split=5,
        min_samples_leaf=2,
        class_weight="balanced_subsample",
        n_jobs=-1,
        random_state=42
    ))
])

# Perform cross-validation
cv_res = cross_validate(
    rf_pipeline,
    X,
    y,
    cv=cv,
    scoring=scoring,
    n_jobs=-1,
    return_train_score=False
)

# Summarize results
res_df = pd.DataFrame([{
    "Model": "RandomForest",
    "CV_Accuracy": np.mean(cv_res["test_accuracy"]),
    "CV_MacroF1": np.mean(cv_res["test_macro_f1"])
}])

print(res_df)


          Model  CV_Accuracy  CV_MacroF1
0  RandomForest         0.68    0.624086


**Ensemble**

In [ ]:
base_learners = [
    ("logreg", Pipeline([
        ("prep", preprocess),
        ("select", feature_selection),
        ("clf", LogisticRegression(
            penalty="elasticnet", solver="saga", class_weight="balanced",
            l1_ratio=0.5, C=0.5, max_iter=1000, n_jobs=-1, random_state=42
        ))
    ])),
    ("svm", Pipeline([
        ("prep", preprocess),
        ("select", feature_selection),
        ("clf", SVC(kernel="rbf", C=0.6, gamma="scale", probability=True, random_state=42))
    ])),
    ("tree", Pipeline([
        ("prep", preprocess),
        ("select", feature_selection),
        ("clf", DecisionTreeClassifier(
            criterion="gini", max_depth=20, min_samples_split=10,
            min_samples_leaf=5, random_state=42
        ))
    ])),
    ("rf", Pipeline([
        ("select", feature_selection),
        ("clf", RandomForestClassifier(
            n_estimators=300, max_depth=None, min_samples_split=5,
            min_samples_leaf=2, class_weight="balanced_subsample",
            n_jobs=-1, random_state=42
        ))
    ]))
]

# stacking ensemble
meta_model = LogisticRegression(solver="lbfgs", max_iter=1000, class_weight="balanced", random_state=42)
stacked_clf = StackingClassifier(
    estimators=base_learners,
    final_estimator=meta_model,
    stack_method="predict_proba",  
    n_jobs=-1,
    passthrough=False               
)

#cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = {
    "accuracy": make_scorer(accuracy_score),
    "macro_f1": make_scorer(f1_score, average="macro")
}

cv_res = cross_validate(
    stacked_clf,
    X, y,
    cv=cv,
    scoring=scoring,
    n_jobs=-1,
    return_train_score=False
)

res_df = pd.DataFrame([{
    "Model": "Stacked Ensemble",
    "CV_Accuracy": np.mean(cv_res["test_accuracy"]),
    "CV_MacroF1": np.mean(cv_res["test_macro_f1"])
}])

print("\nStacked Ensemble Performance:")
print(res_df.to_string(index=False))

### Visual Analysis

In [ ]:
unique_counts = df.nunique()
threshold = 16

unique_counts_grouped = unique_counts.apply(lambda x: x if x <= threshold else threshold + 1)
value_distribution = unique_counts_grouped.value_counts().sort_index()

# rename the last bin to ">threshold"
value_distribution.index = [
    str(i) if i <= threshold else f">{threshold}" for i in value_distribution.index
]

plt.figure(figsize=(8, 5))
plt.bar(value_distribution.index, value_distribution.values, color="steelblue")
plt.xlabel("Number of unique values per feature")
plt.ylabel("Number of features")
plt.title("Distribution of feature uniqueness (grouped)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

df.info()
df.describe().T

df["Class"].value_counts().plot(kind="bar", figsize=(12, 4), color="salmon")
plt.title("Reviews per Author")
plt.ylabel("Count")
plt.show()

df.var().sort_values().head(10)  # lowest variance features
df.var().sort_values(ascending=False).head(10)  # highest variance

sampled = df.sample(axis=1, n=20, random_state=42)
corr = sampled.corr()
plt.imshow(corr, cmap="coolwarm", vmin=-1, vmax=1)
plt.colorbar()
plt.title("Correlation between 20 random features")
plt.show()

df.groupby("class").mean().T.var(axis=1).sort_values(ascending=False).head(10)

feature = "V1234"  # choose one from above
df.boxplot(column=feature, by="Class", figsize=(10, 4))
plt.title(f"Distribution of {feature} across authors")
plt.suptitle("")
plt.show()


In [ ]:
importance = np.mean(np.abs(coef), axis=0)
top_idx = np.argsort(importance)[-20:]  # top 20
plt.figure(figsize=(10, 5))
plt.barh(np.array(feature_names)[top_idx], importance[top_idx], color='tomato')
plt.title("Top 20 most influential features (Logistic Regression)")
plt.xlabel("Mean absolute coefficient")
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X_scaled = StandardScaler().fit_transform(X)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(8, 6))
for name in le.classes_:
    mask = df["Class"] == name
    plt.scatter(X_pca[mask, 0], X_pca[mask, 1], alpha=0.6, label=name)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.title("PCA projection of review features by author")
plt.show()

In [ ]:
# Logistic Regression importance (mean absolute coefficient)
log_reg_importance = np.mean(np.abs(model.coef_), axis=0)

# LightGBM importance (gain-based)
lgb_importance = lgb.feature_importances_

# Combine into a dataframe
importance_df = pd.DataFrame({
    "feature": X.columns,
    "log_reg": log_reg_importance,
    "lightgbm": lgb_importance
})

# Normalize both scales for comparison
importance_df["log_reg_norm"] = importance_df["log_reg"] / importance_df["log_reg"].max()
importance_df["lightgbm_norm"] = importance_df["lightgbm"] / importance_df["lightgbm"].max()


In [ ]:
# Correlation between importances
corr = importance_df[["log_reg_norm", "lightgbm_norm"]].corr().iloc[0, 1]
print("Correlation between model importances:", round(corr, 3))

# Scatter plot
plt.figure(figsize=(6, 6))
plt.scatter(importance_df["log_reg_norm"], importance_df["lightgbm_norm"], alpha=0.5, color='tomato')
plt.xlabel("Logistic Regression importance (normalized)")
plt.ylabel("LightGBM importance (normalized)")
plt.title(f"Feature Importance Correlation (r={corr:.2f})")
plt.show()

# Show top features by either model
top = importance_df.sort_values("lightgbm_norm", ascending=False).head(20)
top.plot(x="feature", y=["log_reg_norm", "lightgbm_norm"], kind="barh", figsize=(10, 6), color=["#e07b7b", "#7bb8e0"])
plt.title("Top 20 Features by LightGBM vs Logistic Regression")
plt.xlabel("Normalized Importance")
plt.tight_layout()
plt.show()


In [ ]:
# Sort by both models' normalized importance
importance_df_sorted = importance_df.sort_values("lightgbm_norm", ascending=False)

# Compute difference between models
importance_df_sorted["diff"] = importance_df_sorted["lightgbm_norm"] - importance_df_sorted["log_reg_norm"]

# Features LightGBM finds much more important
strong_lgb = importance_df_sorted.nlargest(15, "diff")[["feature", "lightgbm_norm", "log_reg_norm", "diff"]]

# Features Logistic Regression finds more important
strong_log = importance_df_sorted.nsmallest(15, "diff")[["feature", "lightgbm_norm", "log_reg_norm", "diff"]]

# Plot LightGBM-dominant features
plt.figure(figsize=(10, 6))
plt.barh(strong_lgb["feature"], strong_lgb["lightgbm_norm"], color="#7bb8e0", label="LightGBM")
plt.barh(strong_lgb["feature"], strong_lgb["log_reg_norm"], color="#e07b7b", alpha=0.6, label="LogReg")
plt.title("Features LightGBM finds more important")
plt.xlabel("Normalized Importance")
plt.legend()
plt.tight_layout()
plt.show()

# Plot Logistic Regression-dominant features
plt.figure(figsize=(10, 6))
plt.barh(strong_log["feature"], strong_log["log_reg_norm"], color="#e07b7b", label="LogReg")
plt.barh(strong_log["feature"], strong_log["lightgbm_norm"], color="#7bb8e0", alpha=0.6, label="LightGBM")
plt.title("Features Logistic Regression finds more important")
plt.xlabel("Normalized Importance")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.inspection import PartialDependenceDisplay

# Example: visualize partial dependence of feature 'V6567' for one author class (e.g., 'Riley')
target_class = list(le.classes_).index('Riley')  # change 'Riley' to any author name

PartialDependenceDisplay.from_estimator(
    lgb,
    X,
    ['V6567'],
    target=target_class,
    grid_resolution=50
)


### Stacking

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
X = df.drop(columns=["ID", "Class"])
y = df["Class"]

le = LabelEncoder()
y_enc = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc, test_size=0.2, random_state=42
)


In [ ]:
lgb = LGBMClassifier(
    n_estimators=1000,
    learning_rate=0.05,
    num_leaves=31,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

logreg = LogisticRegression(max_iter=1000, n_jobs=-1)


In [ ]:
stack = StackingClassifier(
    estimators=[
        ('lgbm', lgb),
        ('logreg', logreg)
    ],
    final_estimator=LogisticRegression(max_iter=1000),
    stack_method='predict_proba',  # use class probabilities as input to meta model
    n_jobs=-1
)


In [ ]:
stack.fit(X_train, y_train)
y_pred = stack.predict(X_test)

print("Stacked model accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))


In [ ]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

print("Stacked model accuracy:", accuracy_score(y_test, y_pred))

unique_labels = np.unique(y_test)  # only labels present in test data
print(classification_report(
    y_test,
    y_pred,
    labels=unique_labels,
    target_names=le.inverse_transform(unique_labels)
))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Get the final (meta) model
meta_model = stack.final_estimator_

coefs = np.abs(meta_model.coef_).mean(axis=0)

base_model_names = [name for name, _ in stack.estimators_]
n_classes = meta_model.coef_.shape[0]
split_size = int(len(coefs) / len(base_model_names))
weights = [coefs[i * split_size:(i + 1) * split_size].mean() for i in range(len(base_model_names))]

# Normalize weights
weights = np.array(weights) / np.sum(weights)
